# Web scraping

In [145]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
def anvndata():
    """Ask for user preferences"""
    uppk = input("\nDo you want to update [U] or clear [C] calender? ").lower().startswith("u") 
    
    if uppk:
        se_aktiv = input("\nDo you want to be reminded of solar eclipses? Enter Yes or No: ").lower().startswith("y")
        fm_aktiv = input("\nDo you want to be reminded of upcoming full moons? Enter Yes or No: ").lower().startswith("y")
        nl_aktiv = input("\nDo you want to be reminded of dates with northern light? Enter Yes or No: ").lower().startswith("y")
        
        rem_prio = ""
        while (se_aktiv or fm_aktiv or nl_aktiv) and (not rem_prio):
            rem_prio = input("\nHow many hours before an upcoming event should I be reminded: ")
            rem_prio = int(rem_prio) if rem_prio.isnumeric() else ""
        return (True,se_aktiv,fm_aktiv,nl_aktiv,rem_prio)
    else:
        return (False,False,False,False,0)

In [3]:
def getnorthlgt():
    return []

In [4]:
def getsoleclip(idstr):
    ut=[]
    pattern = r"[\d-]+"
    url = "https://www.astroinfo.se/formorkelser/solformorkelser/"
    mc = "div.entry.themeform>p"
    soup = BeautifulSoup(requests.get(url).content)
    
    for i in soup.select(mc):
        tmp = i.select("a")[0]
        date = re.findall(pattern, tmp.text)[0]
        tist = datetime.datetime.strptime(date + " 08:00", '%Y-%m-%d %H:%M')
        tien = datetime.datetime.strptime(date + " 12:00", '%Y-%m-%d %H:%M')
        desc = re.sub(pattern,"", tmp.text).strip()
        ref = tmp["href"]
        ut.append((tist,tien,desc,desc,ref))
    return ut

In [5]:
def getfullmoon(idstr):
    ut=[]
    pattern = r"[\d-]+"
    manad = [("januari","1"),("februari","2"),("mars","3"),("april","4"),("maj","5"),("juni","6"),("juli","7"),
             ("augusti","8"),("september","9"),("oktober","10"),("november","11"),("december","12")]
    url = "https://www.kalender-365.se/manen/manens-lagen.html"
    mc = "div.full_moon_phases_div.left_moon_phases"
    y = datetime.datetime.now().year
    soup = BeautifulSoup(requests.get(url).content)
    
    for i in soup.select(mc):
        if str(y) in i.select("h2")[0].text:
            for j in i.select("tbody>tr"):
                desc = j.select("td:nth-child(1)")[0].text
                date = j.select("td:nth-child(2)")[0].text
                for svm, nrm in manad:
                    if svm in date:
                        date = date.replace(svm,nrm)
                tist = datetime.datetime.strptime(date + " " + j.select("td:nth-child(3)")[0].text, '%d %m %Y %H:%M:%S')
                tien = tist + datetime.timedelta(hours=1)
                if "Fullmåne" in desc:
                    ut.append((tist,tien,desc,desc,""))
    return ut

In [6]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
#SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
SCOPES = ['https://www.googleapis.com/auth/calendar']


def get_calendar_service():
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    service = build('calendar', 'v3', credentials=creds)
    return service

In [7]:
def egenkalbort(idstr):
    service = get_calendar_service()  
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    events_result = service.events().list(calendarId="primary", timeMin=now, singleEvents=True,
                                          orderBy="startTime").execute()
    events = events_result.get('items', [])

    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
         #if "summary" in event:
        if idstr in event["summary"]:
            try:
                service.events().delete(calendarId='primary',eventId=event["id"]).execute()
            except:
                print("Failed to delete event")

In [8]:
def egenklany(poster,forv,idstr):
    service = get_calendar_service() 
    
    for tst, tend, title, des, link in poster:
        if tst > datetime.datetime.now():
            lank = f"\n <a href='{link}'>Länk</a>" if link else ""
            event_result = service.events().insert(calendarId='primary',body={
                "summary": title + "      id=" + idstr,
                "description": des + lank,
                "start": {"dateTime": tst.isoformat(), "timeZone": "Europe/Stockholm"},
                "end": {"dateTime": tend.isoformat(), "timeZone": "Europe/Stockholm"},
                "colorId": 3,
                "reminders":{"overrides":[{"minutes":forv*60,"method":"popup"}],"useDefault":False}}
                                              ).execute()

In [11]:
import re
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup

idstr = "AUT123aaB7"
uppk,se_aktiv,fm_aktiv,nl_aktiv,rem_prio = anvndata()

if uppk:
    poster = []
    kh1 = getsoleclip(idstr) if se_aktiv else []
    kh2 = getfullmoon(idstr) if fm_aktiv else []
    kh3 = getnorthlgt(idstr) if nl_aktiv else []
    poster = kh1 + kh2 + kh3
    
    egenkalbort(idstr)
    egenklany(poster,rem_prio,idstr)
else:
    pass
    egenkalbort(idstr)


Do you want to update [U] or clear [C] calender? u

Do you want to be reminded of solar eclipses? Enter Yes or No: y

Do you want to be reminded of upcoming full moons? Enter Yes or No: y

Do you want to be reminded of dates with northern light? Enter Yes or No: n

How many hours before an upcoming event should I be reminded: 24
